**Nome:** Imario Almeida Barbosa

**Matrícula:** 412976

**Curso:** Engenharia de Computação

#Lista de Exercícios 07

1. Use validação holdout para obter o melhor valor de k para o kNN com e sem escalonamento de features. Para isso, divida o conjunto de treino (80%) entre treino_holdout (70% do conjunto de treino) e validacao_holdout (30% do conjunto de treino). Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação (validacao_holdout). Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn import pipeline
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('/content/drive/MyDrive/UFC/2021.1/ML/dataset/diabetes.csv')

Divisão do conjunto entre treino e teste, sem escalonamento de features

In [3]:
X = df.drop(["Outcome"], axis=1)
y = df["Outcome"]
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

Divisão do conjunto de treino(80%) entre treino_holdout(70%) e validacao_holdout(30%)

In [4]:
treino_holdout = X_treino.sample(frac=0.7, random_state=42)
y_treino_holdout = y_treino.sample(frac=0.7, random_state=42)
validacao_holdout = X_treino.drop(treino_holdout.index)
y_validacao = y_treino.drop(y_treino_holdout.index)

Definição da função para gerar k modelos do Knn

In [19]:
def knN_n_neighbors(X_treino, y_treino, X_teste, y_teste, k):
  acc = []
  ks = np.arange(1,k+1)
  #Criação do modelo
  models = map(lambda x: KNeighborsClassifier(n_neighbors=x), ks)
  set_models = list(models)
  it = 1
  for i in set_models:
    i.fit(X_treino,y_treino)
    #Obtendo as predições
    y_pred = i.predict(X_teste)
    #Calculo da acuracia
    accuracy = {'Scaler': 'No','modelo': f' k = {it}', 'accuracy': metrics.f1_score(y_teste, y_pred, average='weighted', zero_division=0)}
    acc.append(accuracy)
    it += 1
  return acc

Gridsearch para encontrar o melhor k, com o conjunto sem escalonamento

In [20]:
acc = knN_n_neighbors(treino_holdout, y_treino_holdout, validacao_holdout, y_validacao, 25)
grid_search = pd.DataFrame(acc, columns=[ 'Scaler', 'modelo', 'accuracy'])
#grid_search.sort_values(by='accuracy', ascending=False)

Modelo final, conjunto sem escalonamento

In [23]:
supreme_model = KNeighborsClassifier(n_neighbors=9)
supreme_model.fit(X_treino, y_treino);
pred = supreme_model.predict(X_teste)
metrics.f1_score(y_teste, pred, average='weighted', zero_division=0)

0.7240153679944256

Escalonando features com Pipeline

In [24]:
def Knn_pipeline(X_treino, y_treino, X_teste, y_teste, k):
  acc = []
  stds = preprocessing.StandardScaler()
  for i in range(1, k+1):
    pipe = pipeline.Pipeline([('scaler', stds), ('Knn', KNeighborsClassifier(n_neighbors=i))])
    pipe.fit(X_treino,y_treino)
    y_pred = pipe.predict(X_teste)
    accuracy = {'modelo': f'k = {i}', 'accuracy': metrics.f1_score(y_teste, y_pred, average='weighted', zero_division=0), 'Scaler': 'Yes'}
    acc.append(accuracy)
  return acc

Gridsearch com escalonamento

In [25]:
std_Knn = Knn_pipeline(treino_holdout, y_treino_holdout, validacao_holdout, y_validacao, 25)
grid = pd.DataFrame(std_Knn, columns=['Scaler','modelo', 'accuracy'])
grid_result = pd.concat([grid_search, grid])
grid_result.sort_values(by='accuracy', ascending=False)[0:20]

,Scaler,modelo,accuracy
6,Yes,k = 7,0.825278
8,Yes,k = 9,0.822459
7,Yes,k = 8,0.817677
5,Yes,k = 6,0.816275
10,Yes,k = 11,0.805235
12,Yes,k = 13,0.797754
14,Yes,k = 15,0.792814
16,Yes,k = 17,0.791435
18,Yes,k = 19,0.789962
4,Yes,k = 5,0.787560


Criação do modelo final

In [26]:
stds = preprocessing.StandardScaler()
pipe = pipeline.Pipeline([('scaler', stds), ('Knn', KNeighborsClassifier(n_neighbors=7))])
pipe.fit(X_treino,y_treino)
y_pred = pipe.predict(X_teste)
metrics.f1_score(y_teste, y_pred, average='weighted', zero_division=0)

0.676438550793275

2. Use validação cruzada com k-Fold=5 para obter o melhor valor de k para o kNN com e sem escalonamento de features. Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação. Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

Dicas: (1) use a função cross_val_score do scikit learn com cv=5. (2) use o Pipeline do scikit learn para realizar o escalonamento de features. (3) Não use nessa questão o GridSearchCV do scikit learn.

Criação da função para gerar k modelos com crossvalidação, sem escalonamento

In [15]:
def Knn_crossvalidation_(X_treino, y_treino, X_teste, y_teste, k):
  acc = [];
  ks = np.arange(1,k+1);
  #Criação do modelo
  models = map(lambda x: KNeighborsClassifier(n_neighbors=x), ks);
  set_models = list(models);
  it = 1
  for i in set_models:
    cross_result = cross_val_score(i, X_treino, y_treino,cv=5, scoring='f1_macro');
    accuracy = {'Scaler': 'No','modelo': f' k = {it}', 'accuracy': cross_result.mean()};
    acc.append(accuracy);
    it += 1;
  return acc

Criação da primeira parte do Gridsearch

In [16]:
acc = Knn_crossvalidation_(X_treino, y_treino, X_teste, y_teste, 25);
cross_grid_search = pd.DataFrame(acc, columns=[ 'Scaler', 'modelo', 'accuracy'])

Criação da função para criar k modelos com crossvalidação e escalonamento

In [17]:
def Knn_crossvalidation_std(X_treino, y_treino, X_teste, y_teste, k):
  acc = []
  stds = preprocessing.StandardScaler()
  for i in range(1, k+1):
    pipe = pipeline.Pipeline([('scaler', stds), ('Knn', KNeighborsClassifier(n_neighbors=i))])
    cross_result = cross_val_score(pipe, X_treino, y_treino,cv=5, scoring='f1_macro');
    accuracy = {'modelo': f'k = {i}', 'accuracy': cross_result.mean(), 'Scaler': 'Yes'}
    acc.append(accuracy)
  return acc

Criação do gridsearch final

In [18]:
std_crossval = Knn_crossvalidation_std(X_treino, y_treino, X_teste, X_teste, 25)
cross_grid = pd.DataFrame(std_crossval, columns=['Scaler','modelo', 'accuracy'])
grid_result = pd.concat([cross_grid_search, cross_grid])
grid_result.sort_values(by='accuracy', ascending=False)[0:20]

,Scaler,modelo,accuracy
10,Yes,k = 11,0.715437
4,Yes,k = 5,0.711413
5,Yes,k = 6,0.708523
24,Yes,k = 25,0.704590
8,Yes,k = 9,0.702084
6,Yes,k = 7,0.701837
20,Yes,k = 21,0.701444
14,Yes,k = 15,0.700598
22,Yes,k = 23,0.700415
18,Yes,k = 19,0.700141


Criação do modelo final

In [ ]:
#Escalonamento
stds = preprocessing.StandardScaler()
X_treino_std = stds.fit_transform(X_treino)
X_teste_std = stds.transform(X_teste)

#Criação do modelo
final_model = KNeighborsClassifier(n_neighbors=11)
final_model.fit(X_treino_std, y_treino);

#Predição e avaliação
pred = final_model.predict(X_teste_std)
metrics.f1_score(y_teste, pred)

0.5794392523364486

3. Responda a questão 1 só que agora usando o GridSearchCV do scikit learn. Dica: Nesta questão você não deve usar o cross_val_score do scikit learn.


In [ ]:
from sklearn.model_selection import GridSearchCV

Gridsearch sem escalonamento

In [ ]:
parameters = {'n_neighbors': range(1,20)}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters,n_jobs=-1, scoring='f1_macro')
clf.fit(treino_holdout, y_treino_holdout);

In [ ]:
clf.best_score_

0.7118122176024697

In [ ]:
clf.best_params_

{'n_neighbors': 7}

Gridsearch com escalonamento

In [ ]:
stds = preprocessing.StandardScaler()
pipe = pipeline.Pipeline([('scaler', stds), ('Knn', KNeighborsClassifier())])
parameters = {'Knn__n_neighbors': range(1,20)}
clf = GridSearchCV(pipe,parameters, n_jobs=-1,scoring='f1_macro')
clf.fit(treino_holdout, y_treino_holdout);

In [ ]:
clf.best_score_

0.6841408659671447

In [ ]:
clf.best_params_

{'Knn__n_neighbors': 3}

Criação do modelo final

In [27]:
final_model = KNeighborsClassifier(n_neighbors=7)
final_model.fit(X_treino, y_treino);
pred = final_model.predict(X_teste)
metrics.f1_score(y_teste, y_pred, average='weighted', zero_division=0)

0.676438550793275